# Notebook that processes the activitysim run log files

In [1]:
import io
import os
import numpy as np
import pandas as pd

## I/O

In [2]:
root_dir = r"\\corp.pbwan.net\us\CentralData\DCCLDA00\Standard\sag\projects\AMPO\31000188\Phase 9A\benchmarking"

working_dir = r"C:\Users\wangs1\OneDrive - WSP O365\2022_P_31000188.000_AMPO\Phase 9\Task 1 benchmarking"

output_dir = os.path.join(working_dir, "data", "processed")

In [5]:
run_dictionary = {
    "mtc extended" : {
        # "CS Sharrow Off 2-14" : {
        #     "sample rate" : 100,
        #     "sharrow" : False,
        #     "code version" : "v1.3beta",
        #     "input_dir" : r"\\corp.pbwan.net\us\CentralData\DCCLDA00\Standard\sag\projects\AMPO\31000188\Phase 9A\benchmarking\from_cs\sharrow-off"
        # },
        # "CS Sharrow On 2-14" : {
        #     "sample rate" : 100,
        #     "sharrow" : True,
        #     "code version" : "v1.3beta",
        #     "input_dir" : r"\\corp.pbwan.net\us\CentralData\DCCLDA00\Standard\sag\projects\AMPO\31000188\Phase 9A\benchmarking\from_cs\sharrow-on"
        # },
        "WSP Sharrow Off 2-14" : {
            "sample rate" : 100,
            "sharrow" : False,
            "code version" : "v1.3beta",
            "input_dir" : r"\\corp.pbwan.net\us\CentralData\DCCLDA00\Standard\sag\projects\AMPO\31000188\Phase 9A\benchmarking\2-14-2024\Tier 2\output-2024-02-14-140048\output-legacy\log"
        },
        "WSP Sharrow On 2-14" : {
            "sample rate" : 100,
            "sharrow" : True,
            "code version" : "v1.3beta",
            "input_dir" : r"\\corp.pbwan.net\us\CentralData\DCCLDA00\Standard\sag\projects\AMPO\31000188\Phase 9A\benchmarking\2-14-2024\Tier 1\output-2024-02-14-153559\output-sharrow\log"
        },
        "WSP Sharrow Off 2-21" : {
            "sample rate" : 100,
            "sharrow" : False,
            "code version" : "ff3e96b",
            "input_dir" : r"\\corp.pbwan.net\us\CentralData\DCCLDA00\Standard\sag\projects\AMPO\31000188\Phase 9A\benchmarking\2-20-2024\Tier 2 Pool B\output-2024-02-20-195915\output-legacy\log"
        },
        "WSP Sharrow On 2-21" : {
            "sample rate" : 100,
            "sharrow" : True,
            "code version" : "ff3e96b",
            "input_dir" : r"\\corp.pbwan.net\us\CentralData\DCCLDA00\Standard\sag\projects\AMPO\31000188\Phase 9A\benchmarking\2-21-2024\Tier 2 Pool A\output-2024-02-21-113615\output-sharrow\log"
        },
        "WSP Sharrow Off 2-28" : {
            "sample rate" : 100,
            "sharrow" : False,
            "code version" : "64cacd2a",
            "input_dir" : r"\\corp.pbwan.net\us\CentralData\DCCLDA00\Standard\sag\projects\AMPO\31000188\Phase 9A\benchmarking\2-28-2024\Tier 2 Pool B\output-2024-02-28-153456\output-legacy\log"
        },
        "WSP Sharrow On 3-4" : {
            "sample rate" : 100,
            "sharrow" : True,
            "code version" : "2f9afa0a",
            "input_dir" : r"\\corp.pbwan.net\us\CentralData\DCCLDA00\Standard\sag\projects\AMPO\31000188\Phase 9A\benchmarking\3-4-2024\Tier 2 Pool A\output-2024-03-04-182100\output-sharrow\log"
        },
    }
}

## Process log files

In [6]:
pattern = '%d/%m/%Y'

import datetime

def starts_with_time(text):
    try:
        datetime.datetime.strptime(text[:10], pattern)
        return True
    except ValueError:
        return False


In [7]:
activitysim_log_df = pd.DataFrame()
memory_df = pd.DataFrame()
time_log_df = pd.DataFrame()

for model, runs in run_dictionary.items():
    for run, run_specs in runs.items():
        
        # get input directory path
        if "input_dir" in run_specs:
            input_dir = run_specs["input_dir"]
        else:
            input_dir = os.path.join(root_dir, model, run)

        # check if input directory exists
        if not os.path.exists(input_dir):
            print(f"Input directory {input_dir} does not exist.")
            continue

        # activitysim log file
        temp_activitysim_log_file = os.path.join(input_dir, "activitysim.log")
        with open(temp_activitysim_log_file, "r") as file:
            lines = []
            for line in file:
                if starts_with_time(line):
                    lines.append(line)
        temp_activitysim_log_df = pd.read_csv(io.StringIO('\n'.join(lines)), sep=" - INFO - | - WARNING - | - DEBUG - | - NOTIFY - | - ERROR -", names=["time", "details"])
        temp_activitysim_log_df["time"] = pd.to_datetime(temp_activitysim_log_df["time"], format="%d/%m/%Y %H:%M:%S.%f")

        print(temp_activitysim_log_df.shape)

        # add run spec
        temp_activitysim_log_df['model'] = model
        temp_activitysim_log_df['run'] = run
        # temp_activitysim_log_df['run_name'] = f"{model} {spec_value} pct sample"
        
        # loop through key and value from spec dictionary
        for spec_key, spec_value in run_specs.items():
            temp_activitysim_log_df[spec_key] = spec_value

        activitysim_log_df = activitysim_log_df.append(temp_activitysim_log_df, ignore_index = True)

        # memory profile csv
        temp_memory_profile_file = os.path.join(input_dir, "memory_profile.csv")
        temp_memory_df = pd.read_csv(temp_memory_profile_file)
        temp_memory_df['time'] = pd.to_datetime(temp_memory_df['time'])

        # add run spec
        temp_memory_df['model'] = model
        temp_memory_df['run'] = run
        # temp_memory_df['run_name'] = f"{model} {spec_value} pct sample"
        for spec_key, spec_value in run_specs.items():
            temp_memory_df[spec_key] = spec_value

        memory_df = memory_df.append(temp_memory_df, ignore_index = True)

        # activitysim time log file
        temp_time_log_file = os.path.join(input_dir, "timing_log.csv")
        temp_time_log_df = pd.read_csv(temp_time_log_file)

        time_log_df = time_log_df.append(temp_time_log_df, ignore_index = True)

C:\Users\wangs1\AppData\Local\Temp\ipykernel_26416\96564015.py:26: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  temp_activitysim_log_df = pd.read_csv(io.StringIO('\n'.join(lines)), sep=" - INFO - | - WARNING - | - DEBUG - | - NOTIFY - | - ERROR -", names=["time", "details"])
C:\Users\wangs1\AppData\Local\Temp\ipykernel_26416\96564015.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  activitysim_log_df = activitysim_log_df.append(temp_activitysim_log_df, ignore_index = True)


(1687, 2)


C:\Users\wangs1\AppData\Local\Temp\ipykernel_26416\96564015.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  memory_df = memory_df.append(temp_memory_df, ignore_index = True)
C:\Users\wangs1\AppData\Local\Temp\ipykernel_26416\96564015.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  time_log_df = time_log_df.append(temp_time_log_df, ignore_index = True)
C:\Users\wangs1\AppData\Local\Temp\ipykernel_26416\96564015.py:26: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  temp_activitysim_log_df = pd.read_csv(io.StringIO('\n'.join(lines)), sep=" - INFO - | - WARNING - | - DEBUG - | - NOTIFY - | - ERROR -", names=["time"

(5112, 2)


C:\Users\wangs1\AppData\Local\Temp\ipykernel_26416\96564015.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  memory_df = memory_df.append(temp_memory_df, ignore_index = True)
C:\Users\wangs1\AppData\Local\Temp\ipykernel_26416\96564015.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  time_log_df = time_log_df.append(temp_time_log_df, ignore_index = True)
C:\Users\wangs1\AppData\Local\Temp\ipykernel_26416\96564015.py:26: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  temp_activitysim_log_df = pd.read_csv(io.StringIO('\n'.join(lines)), sep=" - INFO - | - WARNING - | - DEBUG - | - NOTIFY - | - ERROR -", names=["time"

...

C:\Users\wangs1\AppData\Local\Temp\ipykernel_26416\96564015.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  memory_df = memory_df.append(temp_memory_df, ignore_index = True)
C:\Users\wangs1\AppData\Local\Temp\ipykernel_26416\96564015.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  time_log_df = time_log_df.append(temp_time_log_df, ignore_index = True)
C:\Users\wangs1\AppData\Local\Temp\ipykernel_26416\96564015.py:26: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  temp_activitysim_log_df = pd.read_csv(io.StringIO('\n'.join(lines)), sep=" - INFO - | - WARNING - | - DEBUG - | - NOTIFY - | - ERROR -", names=["time"

(4751, 2)


C:\Users\wangs1\AppData\Local\Temp\ipykernel_26416\96564015.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  memory_df = memory_df.append(temp_memory_df, ignore_index = True)
C:\Users\wangs1\AppData\Local\Temp\ipykernel_26416\96564015.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  time_log_df = time_log_df.append(temp_time_log_df, ignore_index = True)
C:\Users\wangs1\AppData\Local\Temp\ipykernel_26416\96564015.py:26: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  temp_activitysim_log_df = pd.read_csv(io.StringIO('\n'.join(lines)), sep=" - INFO - | - WARNING - | - DEBUG - | - NOTIFY - | - ERROR -", names=["time"

(13296, 2)


C:\Users\wangs1\AppData\Local\Temp\ipykernel_26416\96564015.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  memory_df = memory_df.append(temp_memory_df, ignore_index = True)
C:\Users\wangs1\AppData\Local\Temp\ipykernel_26416\96564015.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  time_log_df = time_log_df.append(temp_time_log_df, ignore_index = True)


In [8]:
activitysim_log_df

,time,details,model,run,sample rate,sharrow,code version,input_dir
0,2024-02-14 14:01:07.653,activitysim - Read logging configuration from:...,mtc extended,WSP Sharrow Off 2-14,100,False,v1.3beta,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...
1,2024-02-14 14:01:07.653,activitysim.cli.run - SETTING configs_dir: (Wi...,mtc extended,WSP Sharrow Off 2-14,100,False,v1.3beta,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...
2,2024-02-14 14:01:07.653,activitysim.cli.run - SETTING settings_file_na...,mtc extended,WSP Sharrow Off 2-14,100,False,v1.3beta,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...
3,2024-02-14 14:01:07.653,activitysim.cli.run - SETTING data_dir: (Windo...,mtc extended,WSP Sharrow Off 2-14,100,False,v1.3beta,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...
4,2024-02-14 14:01:07.653,activitysim.cli.run - SETTING output_dir: outp...,mtc extended,WSP Sharrow Off 2-14,100,False,v1.3beta,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...
...,...,...,...,...,...,...,...,...
45122,2024-03-05 09:57:39.306,activitysim.core.workflow.runner - time to ex...,mtc extended,WSP Sharrow On 3-4,100,True,2f9afa0a,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...
45123,2024-03-05 09:57:43.559,activitysim.core.tracing - Time to execute run...,mtc extended,WSP Sharrow On 3-4,100,True,2f9afa0a,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...
45124,2024-03-05 09:57:43.559,activitysim.core.mem - MainProcess high water ...,mtc extended,WSP Sharrow On 3-4,100,True,2f9afa0a,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...
45125,2024-03-05 09:57:43.559,activitysim.core.mem - MainProcess high water ...,mtc extended,WSP Sharrow On 3-4,100,True,2f9afa0a,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...


In [11]:
# event_list = memory_df['event'].dropna().unique()
# event_list

# activitysim_log_df['event_from_log'] = activitysim_log_df['details'].apply(lambda x: [c for c in event_list if c in x])
# activitysim_log_df['event_from_log'] = activitysim_log_df['event_from_log'].apply(lambda x: x[-1] if len(x)>0 else np.nan)


# event_from_log_ffill = activitysim_log_df['event_from_log'].fillna(method = 'ffill')
# event_from_log_bfill = activitysim_log_df['event_from_log'].fillna(method = 'bfill')

# activitysim_log_df['event_from_log'] = (event_from_log_ffill == event_from_log_bfill) * event_from_log_bfill

# activitysim_log_df['time_from_log'] = activitysim_log_df['time']

# add event (model component to activitysim log)
# when a model component starts, it print out "#run_model running step" in the details

prefix = "#run_model running step"
activitysim_log_df['event_from_log'] = activitysim_log_df['details'].apply(lambda x: x.split(prefix)[-1].strip() if prefix in x else np.nan)
activitysim_log_df['event_from_log'] = activitysim_log_df.apply(lambda x: "open pipeline" if "Read logging configuration" in x['details'] else x['event_from_log'], axis = 1)
activitysim_log_df['event_from_log'] = activitysim_log_df.apply(lambda x: "preload_injectables" if "preload_injectables" in x['details'] else x['event_from_log'], axis = 1)
activitysim_log_df['event_from_log'] = activitysim_log_df['event_from_log'].fillna(method = 'ffill')

In [12]:
activitysim_log_df

,time,details,model,run,sample rate,sharrow,code version,input_dir,event_from_log
0,2024-02-14 14:01:07.653,activitysim - Read logging configuration from:...,mtc extended,WSP Sharrow Off 2-14,100,False,v1.3beta,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...,open pipeline
1,2024-02-14 14:01:07.653,activitysim.cli.run - SETTING configs_dir: (Wi...,mtc extended,WSP Sharrow Off 2-14,100,False,v1.3beta,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...,open pipeline
2,2024-02-14 14:01:07.653,activitysim.cli.run - SETTING settings_file_na...,mtc extended,WSP Sharrow Off 2-14,100,False,v1.3beta,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...,open pipeline
3,2024-02-14 14:01:07.653,activitysim.cli.run - SETTING data_dir: (Windo...,mtc extended,WSP Sharrow Off 2-14,100,False,v1.3beta,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...,open pipeline
4,2024-02-14 14:01:07.653,activitysim.cli.run - SETTING output_dir: outp...,mtc extended,WSP Sharrow Off 2-14,100,False,v1.3beta,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...,open pipeline
...,...,...,...,...,...,...,...,...,...
45122,2024-03-05 09:57:39.306,activitysim.core.workflow.runner - time to ex...,mtc extended,WSP Sharrow On 3-4,100,True,2f9afa0a,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...,write_tables
45123,2024-03-05 09:57:43.559,activitysim.core.tracing - Time to execute run...,mtc extended,WSP Sharrow On 3-4,100,True,2f9afa0a,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...,write_tables
45124,2024-03-05 09:57:43.559,activitysim.core.mem - MainProcess high water ...,mtc extended,WSP Sharrow On 3-4,100,True,2f9afa0a,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...,write_tables
45125,2024-03-05 09:57:43.559,activitysim.core.mem - MainProcess high water ...,mtc extended,WSP Sharrow On 3-4,100,True,2f9afa0a,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...,write_tables


In [13]:
# get number of rows in the chooser table

activitysim_log_df['chooser_rows'] = activitysim_log_df['details'].str.split(' ')

def get_num_rows(list_str):
    if isinstance(list_str, list):
        for i in range(len(list_str)):
            if list_str[i] in ["rows", "chooser", "choosers", "trip", "trips", "tour", "tours"]:
                return list_str[i-1]

activitysim_log_df['chooser_rows'] = activitysim_log_df['chooser_rows'].apply(lambda x: get_num_rows(x) )

In [14]:
activitysim_log_df['chooser_rows_numeric'] = pd.to_numeric(activitysim_log_df['chooser_rows'], errors="coerce")

In [15]:
max_num_row_df = activitysim_log_df.groupby(['model', 'run','event_from_log'])['chooser_rows_numeric'].max().reset_index()
max_num_row_df.rename(columns = {'chooser_rows_numeric' : 'max_chooser_rows'}, inplace = True)

In [16]:
summarize_memory_df = memory_df.groupby(['model', 'run','event'])['time'].agg(
    ["min","max"]
).reset_index().sort_values(by = ['min'], ascending = True)

summarize_memory_df['duration'] = (summarize_memory_df['max'] - summarize_memory_df['min']).dt.total_seconds()

In [17]:
summarize_memory_df

,model,run,event,min,max,duration
7,mtc extended,WSP Sharrow Off 2-14,preload_injectables,2024-02-14 14:01:08.070295,2024-02-14 14:01:08.070295,0.000000
6,mtc extended,WSP Sharrow Off 2-14,input_checker,2024-02-14 14:01:08.086972,2024-02-14 14:01:18.903776,10.816804
5,mtc extended,WSP Sharrow Off 2-14,initialize_proto_population,2024-02-14 14:01:18.920445,2024-02-14 14:02:05.134477,46.214032
2,mtc extended,WSP Sharrow Off 2-14,compute_disaggregate_accessibility,2024-02-14 14:02:05.420954,2024-02-14 14:17:48.845066,943.424112
4,mtc extended,WSP Sharrow Off 2-14,initialize_landuse,2024-02-14 14:17:49.306988,2024-02-14 14:17:49.306988,0.000000
...,...,...,...,...,...,...
178,mtc extended,WSP Sharrow On 3-4,write_data_dictionary,2024-03-05 09:49:28.037438,2024-03-05 09:51:39.827980,131.790542
169,mtc extended,WSP Sharrow On 3-4,track_skim_usage,2024-03-05 09:51:41.221097,2024-03-05 09:51:44.794094,3.572997
180,mtc extended,WSP Sharrow On 3-4,write_trip_matrices,2024-03-05 09:51:45.969884,2024-03-05 09:55:33.560420,227.590536
179,mtc extended,WSP Sharrow On 3-4,write_tables,2024-03-05 09:55:34.740586,2024-03-05 09:57:43.090766,128.350180


In [18]:
activitysim_log_df.to_csv(os.path.join(output_dir, 'activitysim_log_df.csv'),index=False)

In [19]:
activitysim_log_df.head(3)

,time,details,model,run,sample rate,sharrow,code version,input_dir,event_from_log,chooser_rows,chooser_rows_numeric
0,2024-02-14 14:01:07.653,activitysim - Read logging configuration from:...,mtc extended,WSP Sharrow Off 2-14,100,False,v1.3beta,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...,open pipeline,None,NaN
1,2024-02-14 14:01:07.653,activitysim.cli.run - SETTING configs_dir: (Wi...,mtc extended,WSP Sharrow Off 2-14,100,False,v1.3beta,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...,open pipeline,None,NaN
2,2024-02-14 14:01:07.653,activitysim.cli.run - SETTING settings_file_na...,mtc extended,WSP Sharrow Off 2-14,100,False,v1.3beta,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...,open pipeline,None,NaN


In [20]:
max_num_row_df

,model,run,event_from_log,max_chooser_rows
0,mtc extended,WSP Sharrow Off 2-14,auto_ownership_simulate,2875192.0
1,mtc extended,WSP Sharrow Off 2-14,compute_accessibility,1454.0
2,mtc extended,WSP Sharrow Off 2-14,compute_disaggregate_accessibility,50738784.0
3,mtc extended,WSP Sharrow Off 2-14,initialize_households,NaN
4,mtc extended,WSP Sharrow Off 2-14,initialize_landuse,NaN
...,...,...,...,...
180,mtc extended,WSP Sharrow On 3-4,vehicle_type_choice,2598302.0
181,mtc extended,WSP Sharrow On 3-4,workplace_location,40642538.0
182,mtc extended,WSP Sharrow On 3-4,write_data_dictionary,NaN
183,mtc extended,WSP Sharrow On 3-4,write_tables,NaN


## Join the log files

In [22]:
memory_df

,process,pid,rss,full_rss,uss,cpu,event_idx,event,children,time,model,run,sample rate,sharrow,code version,input_dir
0,python.exe,25552,277209088,300023808,209027072,0.0,0,NaN,1,2024-02-14 14:01:08.053566,mtc extended,WSP Sharrow Off 2-14,100,False,v1.3beta,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...
1,python.exe,25552,281403392,304287744,213221376,0.0,1,preload_injectables,1,2024-02-14 14:01:08.070295,mtc extended,WSP Sharrow Off 2-14,100,False,v1.3beta,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...
2,python.exe,25552,285605888,308490240,217423872,0.0,2,input_checker,1,2024-02-14 14:01:08.086972,mtc extended,WSP Sharrow Off 2-14,100,False,v1.3beta,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...
3,python.exe,25552,390541312,413425664,326553600,0.0,2,input_checker,1,2024-02-14 14:01:08.630398,mtc extended,WSP Sharrow Off 2-14,100,False,v1.3beta,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...
4,python.exe,25552,481910784,504795136,428441600,0.0,2,input_checker,1,2024-02-14 14:01:09.160161,mtc extended,WSP Sharrow Off 2-14,100,False,v1.3beta,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349457,python.exe,23860,58526941184,58550001664,51302526976,0.0,40,write_tables,1,2024-03-05 09:57:37.702536,mtc extended,WSP Sharrow On 3-4,100,True,2f9afa0a,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...
349458,python.exe,23860,57841889280,57864687616,51302526976,0.0,40,write_tables,1,2024-03-05 09:57:39.438323,mtc extended,WSP Sharrow On 3-4,100,True,2f9afa0a,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...
349459,python.exe,23860,57730457600,57753518080,51302526976,0.0,40,write_tables,1,2024-03-05 09:57:41.219844,mtc extended,WSP Sharrow On 3-4,100,True,2f9afa0a,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...
349460,python.exe,23860,57730457600,57753518080,51191095296,0.0,40,write_tables,1,2024-03-05 09:57:43.090766,mtc extended,WSP Sharrow On 3-4,100,True,2f9afa0a,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...


In [24]:
# join based on the closest timestamp

merged_df = pd.merge_asof(
    memory_df.sort_values(by = "time", ascending= True),
    activitysim_log_df.sort_values(by = "time", ascending= True),
    on = "time",
    by = ['model', 'run', 'sample rate','sharrow','code version'],
    #tolerance=pd.Timedelta("20ms")
    direction = "nearest"
)

In [25]:
merged_df

,process,pid,rss,full_rss,uss,cpu,event_idx,event,children,time,...,run,sample rate,sharrow,code version,input_dir_x,details,input_dir_y,event_from_log,chooser_rows,chooser_rows_numeric
0,python.exe,25552,277209088,300023808,209027072,0.0,0,NaN,1,2024-02-14 14:01:08.053566,...,WSP Sharrow Off 2-14,100,False,v1.3beta,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...,activitysim.abm.models.input_checker - reading...,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...,input_checker,None,NaN
1,python.exe,25552,281403392,304287744,213221376,0.0,1,preload_injectables,1,2024-02-14 14:01:08.070295,...,WSP Sharrow Off 2-14,100,False,v1.3beta,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...,activitysim.abm.models.input_checker - reading...,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...,input_checker,None,NaN
2,python.exe,25552,285605888,308490240,217423872,0.0,2,input_checker,1,2024-02-14 14:01:08.086972,...,WSP Sharrow Off 2-14,100,False,v1.3beta,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...,activitysim.abm.models.input_checker - reading...,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...,input_checker,None,NaN
3,python.exe,25552,390541312,413425664,326553600,0.0,2,input_checker,1,2024-02-14 14:01:08.630398,...,WSP Sharrow Off 2-14,100,False,v1.3beta,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...,activitysim.abm.models.input_checker - reading...,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...,input_checker,None,NaN
4,python.exe,25552,481910784,504795136,428441600,0.0,2,input_checker,1,2024-02-14 14:01:09.160161,...,WSP Sharrow Off 2-14,100,False,v1.3beta,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...,activitysim.core.input - Reading CSV file data...,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...,input_checker,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349457,python.exe,23860,58526941184,58550001664,51302526976,0.0,40,write_tables,1,2024-03-05 09:57:37.702536,...,WSP Sharrow On 3-4,100,True,2f9afa0a,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...,activitysim.core.workflow.runner - time to ex...,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...,write_tables,None,NaN
349458,python.exe,23860,57841889280,57864687616,51302526976,0.0,40,write_tables,1,2024-03-05 09:57:39.438323,...,WSP Sharrow On 3-4,100,True,2f9afa0a,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...,activitysim.core.workflow.runner - time to ex...,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...,write_tables,None,NaN
349459,python.exe,23860,57730457600,57753518080,51302526976,0.0,40,write_tables,1,2024-03-05 09:57:41.219844,...,WSP Sharrow On 3-4,100,True,2f9afa0a,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...,activitysim.core.workflow.runner - time to ex...,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...,write_tables,None,NaN
349460,python.exe,23860,57730457600,57753518080,51191095296,0.0,40,write_tables,1,2024-03-05 09:57:43.090766,...,WSP Sharrow On 3-4,100,True,2f9afa0a,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...,activitysim.core.mem - MainProcess high water ...,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...,write_tables,None,NaN


In [26]:
merged_df.event.unique()

array([nan, 'preload_injectables', 'input_checker',
       'initialize_proto_population',
       'compute_disaggregate_accessibility', 'initialize_landuse',
       'initialize_households', 'compute_accessibility',
       'school_location', 'workplace_location', 'auto_ownership_simulate',
       'vehicle_type_choice', 'free_parking', 'cdap_simulate',
       'mandatory_tour_frequency', 'mandatory_tour_scheduling',
       'school_escorting', 'joint_tour_frequency',
       'joint_tour_composition', 'joint_tour_participation',
       'joint_tour_destination', 'joint_tour_scheduling',
       'non_mandatory_tour_frequency', 'non_mandatory_tour_destination',
       'non_mandatory_tour_scheduling', 'vehicle_allocation',
       'tour_mode_choice_simulate', 'atwork_subtour_frequency',
       'atwork_subtour_destination', 'atwork_subtour_scheduling',
       'atwork_subtour_mode_choice', 'stop_frequency', 'trip_purpose',
       'trip_destination', 'trip_purpose_and_destination',
       'trip_schedu

In [27]:
merged_df.event_from_log.unique()

array(['input_checker', 'initialize_proto_population',
       'compute_disaggregate_accessibility', 'initialize_landuse',
       'initialize_households', 'compute_accessibility',
       'school_location', 'workplace_location', 'auto_ownership_simulate',
       'vehicle_type_choice', 'free_parking', 'cdap_simulate',
       'mandatory_tour_frequency', 'mandatory_tour_scheduling',
       'school_escorting', 'joint_tour_frequency',
       'joint_tour_composition', 'joint_tour_participation',
       'joint_tour_destination', 'joint_tour_scheduling',
       'non_mandatory_tour_frequency', 'non_mandatory_tour_destination',
       'non_mandatory_tour_scheduling', 'vehicle_allocation',
       'tour_mode_choice_simulate', 'atwork_subtour_frequency',
       'atwork_subtour_destination', 'atwork_subtour_scheduling',
       'atwork_subtour_mode_choice', 'stop_frequency', 'trip_purpose',
       'trip_destination', 'trip_purpose_and_destination',
       'trip_scheduling', 'trip_mode_choice', 'write_d

In [28]:
merged_df = pd.merge(
    merged_df,
    max_num_row_df.rename(columns = {'event_from_log':'event'}),
    how = 'left',
    left_on = ['model', 'run', 'event'],
    right_on = ['model', 'run', 'event']
)

In [29]:
merged_df.head(3)

,process,pid,rss,full_rss,uss,cpu,event_idx,event,children,time,...,sample rate,sharrow,code version,input_dir_x,details,input_dir_y,event_from_log,chooser_rows,chooser_rows_numeric,max_chooser_rows
0,python.exe,25552,277209088,300023808,209027072,0.0,0,NaN,1,2024-02-14 14:01:08.053566,...,100,False,v1.3beta,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...,activitysim.abm.models.input_checker - reading...,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...,input_checker,None,NaN,NaN
1,python.exe,25552,281403392,304287744,213221376,0.0,1,preload_injectables,1,2024-02-14 14:01:08.070295,...,100,False,v1.3beta,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...,activitysim.abm.models.input_checker - reading...,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...,input_checker,None,NaN,NaN
2,python.exe,25552,285605888,308490240,217423872,0.0,2,input_checker,1,2024-02-14 14:01:08.086972,...,100,False,v1.3beta,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...,activitysim.abm.models.input_checker - reading...,\\corp.pbwan.net\us\CentralData\DCCLDA00\Stand...,input_checker,None,NaN,NaN


In [30]:
merged_df['time']

0        2024-02-14 14:01:08.053566
1        2024-02-14 14:01:08.070295
2        2024-02-14 14:01:08.086972
3        2024-02-14 14:01:08.630398
4        2024-02-14 14:01:09.160161
                    ...            
349457   2024-03-05 09:57:37.702536
349458   2024-03-05 09:57:39.438323
349459   2024-03-05 09:57:41.219844
349460   2024-03-05 09:57:43.090766
349461   2024-03-05 09:57:44.283314
Name: time, Length: 349462, dtype: datetime64[ns]

In [31]:
merged_df.shape

(349462, 22)

In [32]:
# add elapse time field to calculate the time from the start of the run
merged_df['elapse_time'] = merged_df['time'] - merged_df.groupby(['model', 'run'])['time'].transform('min')

In [33]:
# add a minute since column to calcualte the minutes of the elapse time
merged_df['elapse_time_minutes'] = merged_df['elapse_time'].dt.total_seconds() / 60

In [34]:
merged_df[['time','run', 'elapse_time', 'elapse_time_minutes']]

,time,run,elapse_time,elapse_time_minutes
0,2024-02-14 14:01:08.053566,WSP Sharrow Off 2-14,0 days 00:00:00,0.000000
1,2024-02-14 14:01:08.070295,WSP Sharrow Off 2-14,0 days 00:00:00.016729,0.000279
2,2024-02-14 14:01:08.086972,WSP Sharrow Off 2-14,0 days 00:00:00.033406,0.000557
3,2024-02-14 14:01:08.630398,WSP Sharrow Off 2-14,0 days 00:00:00.576832,0.009614
4,2024-02-14 14:01:09.160161,WSP Sharrow Off 2-14,0 days 00:00:01.106595,0.018443
...,...,...,...,...
349457,2024-03-05 09:57:37.702536,WSP Sharrow On 3-4,0 days 15:36:26.201136,936.436686
349458,2024-03-05 09:57:39.438323,WSP Sharrow On 3-4,0 days 15:36:27.936923,936.465615
349459,2024-03-05 09:57:41.219844,WSP Sharrow On 3-4,0 days 15:36:29.718444,936.495307
349460,2024-03-05 09:57:43.090766,WSP Sharrow On 3-4,0 days 15:36:31.589366,936.526489


In [35]:
# add a fake date to the elapse time
merged_df['time_fake'] = merged_df['elapse_time'] + pd.to_datetime('2025-01-01')

merged_df['time_fake']

0        2025-01-01 00:00:00.000000
1        2025-01-01 00:00:00.016729
2        2025-01-01 00:00:00.033406
3        2025-01-01 00:00:00.576832
4        2025-01-01 00:00:01.106595
                    ...            
349457   2025-01-01 15:36:26.201136
349458   2025-01-01 15:36:27.936923
349459   2025-01-01 15:36:29.718444
349460   2025-01-01 15:36:31.589366
349461   2025-01-01 15:36:32.781914
Name: time_fake, Length: 349462, dtype: datetime64[ns]

In [36]:
merged_df['elapse_time_minutes']

0           0.000000
1           0.000279
2           0.000557
3           0.009614
4           0.018443
             ...    
349457    936.436686
349458    936.465615
349459    936.495307
349460    936.526489
349461    936.546365
Name: elapse_time_minutes, Length: 349462, dtype: float64

## Write out data for viz

In [37]:
# write out
# drop columns starting with input_dir
merged_df.loc[:, ~merged_df.columns.str.startswith('input_dir')].to_csv(os.path.join(output_dir, 'merged_log_df.csv'),index=False)